<img src=https://raw.githubusercontent.com/autonomio/hyperio/master/logo.png>

## How to recover best model from experiment log?
Due to system error or other reason where scan_object is no longer available, it's still possible to get best model/s using nothing but the experiment log. In the below notebook you will learn exactly how.



In [0]:
##!pip install talos

In [4]:
!pip install git+https://github.com/autonomio/talos

  Cloning https://github.com/autonomio/talos to /tmp/pip-req-build-4w0r5zd3
  Running command git clone -q https://github.com/autonomio/talos /tmp/pip-req-build-4w0r5zd3
  Created wheel for talos: filename=talos-0.6.4-cp36-none-any.whl size=53460 sha256=b5f5f0b6786ce42486225bf9109effae828b6fd2056576c069b835922b856e43
  Stored in directory: /tmp/pip-ephem-wheel-cache-vw9ekc_0/wheels/20/f1/9a/63a4168fd779c183fcc5a8ebe6aa34f1f9bc33eaa558e5461b
Successfully built talos


In [0]:
import keras
keras.__version__
import os

In [0]:
import sys
sys.path.insert(0, './talos/')

import sys
sys.path.insert(0, './wrangle/')

import talos
import wrangle
from keras.models import Sequential
from keras.layers import Dense

First we'll have to perform the `Scan()` experiment to produce the experiment log. Because the experiment log is stored on local machine, interrupted `Scan()` or other reason will not affect its availability. The experiment log is updated after each permutation; it contains an up-to-date record of the experiment.

In [0]:
# load the data
x, y = talos.templates.datasets.iris()
x_train, y_train, x_val, y_val = wrangle.array_split(x, y, .3)

# set the parameter space boundary
p = {'activation':['relu', 'elu'],
     'optimizer': ['Nadam', 'Adam'],
     'losses': ['logcosh'],
     'shapes': ['brick'],
     'first_neuron': [16, 32, 64, 128],
     'hidden_layers':[0, 1, 2, 3],
     'dropout': [.2, .3, .4],
     'batch_size': [20, 30, 40, 50],
     'epochs': [10]}

# define the input model
def iris_model(x_train, y_train, x_val, y_val, params):
    print("\nParams:",params)
    model = Sequential()
    model.add(Dense(params['first_neuron'], input_dim=4, activation=params['activation']))

    talos.utils.hidden_layers(model, params, 3)

    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=params['optimizer'], loss=params['losses'], metrics=['acc'])

    #out = model.fit(x_train, y_train, callbacks=[talos.utils.ExperimentLogCallback('minimal_iris', params)],
    #                 batch_size=params['batch_size'],
    #                 epochs=params['epochs'],
    #                 validation_data=[x_val, y_val],
    #                 verbose=0)
    out = model.fit(x_train, y_train, 
                     batch_size=params['batch_size'],
                     epochs=params['epochs'],
                     validation_data=[x_val, y_val],
                     verbose=0)

    return out, model


In [9]:
# start the experiment
scan_object = talos.Scan(x=x_train,
                         y=y_train,
                         x_val=x_val,
                         y_val=y_val,
                         model=iris_model,
                         experiment_name='reactivate',
                         params=p,
                         round_limit=10)

  0%|          | 0/10 [00:00<?, ?it/s]


Params: {'activation': 'relu', 'batch_size': 40, 'dropout': 0.2, 'epochs': 10, 'first_neuron': 32, 'hidden_layers': 3, 'losses': 'logcosh', 'optimizer': 'Adam', 'shapes': 'brick'}




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.













 10%|█         | 1/10 [00:02<00:19,  2.16s/it]


Params: {'activation': 'relu', 'batch_size': 30, 'dropout': 0.4, 'epochs': 10, 'first_neuron': 16, 'hidden_layers': 3, 'losses': 'logcosh', 'optimizer': 'Adam', 'shapes': 'brick'}


 20%|██        | 2/10 [00:03<00:15,  1.95s/it]

...

 80%|████████  | 8/10 [00:10<00:02,  1.21s/it]


Params: {'activation': 'relu', 'batch_size': 50, 'dropout': 0.4, 'epochs': 10, 'first_neuron': 32, 'hidden_layers': 0, 'losses': 'logcosh', 'optimizer': 'Adam', 'shapes': 'brick'}


 90%|█████████ | 9/10 [00:11<00:01,  1.09s/it]


Params: {'activation': 'elu', 'batch_size': 30, 'dropout': 0.4, 'epochs': 10, 'first_neuron': 128, 'hidden_layers': 2, 'losses': 'logcosh', 'optimizer': 'Nadam', 'shapes': 'brick'}


100%|██████████| 10/10 [00:12<00:00,  1.19s/it]


Now we can assume the case where we no longer have access to the `scan_object`. In this `Scan(...experiment_name...)` was set to "reactivate" so we'll find a folder with that name in the present working directory. Next we have to find out what is the name of the experiment log.

In [10]:
# get the name of the experiment log
!ls -lhtr reactivate

total 8.0K
-rw-r--r-- 1 root root    0 Oct 26 10:09 102619100917.csv
-rw-r--r-- 1 root root    0 Oct 26 10:11 102619101134.csv
-rw-r--r-- 1 root root    0 Oct 26 10:12 102619101222.csv
-rw-r--r-- 1 root root    0 Oct 26 10:44 102619104409.csv
-rw-r--r-- 1 root root    0 Oct 26 10:44 102619104429.csv
-rw-r--r-- 1 root root 1.4K Oct 26 11:00 102619105948.csv
-rw-r--r-- 1 root root 1.3K Oct 26 11:06 102619110642.csv


In this case it will be the most recent one so let's go ahead and recover the best models.

In [15]:
from talos.utils.recover_best_model import recover_best_model
results, models = recover_best_model(x_train=x_train,
                                     y_train=y_train,
                                     x_val=x_val,
                                     y_val=y_val,
                                     experiment_log='reactivate/102619110642.csv',
                                     input_model=iris_model,
                                     n_models=10,
                                     task='multi_label')


Params: {'round_epochs': 10, 'val_loss': 0.04402541990081469, 'loss': 0.05175705573388508, 'acc': 0.8095238038471767, 'activation': 'relu', 'batch_size': 30, 'dropout': 0.2, 'epochs': 10, 'first_neuron': 128, 'hidden_layers': 0, 'losses': 'logcosh', 'optimizer': 'Nadam', 'shapes': 'brick'}

Params: {'round_epochs': 10, 'val_loss': 0.009391499993701776, 'loss': 0.055485859513282776, 'acc': 0.7523809586252485, 'activation': 'elu', 'batch_size': 30, 'dropout': 0.4, 'epochs': 10, 'first_neuron': 128, 'hidden_layers': 2, 'losses': 'logcosh', 'optimizer': 'Nadam', 'shapes': 'brick'}

Params: {'round_epochs': 10, 'val_loss': 0.07680413126945496, 'loss': 0.09249024944646017, 'acc': 0.5142857205300104, 'activation': 'relu', 'batch_size': 50, 'dropout': 0.4, 'epochs': 10, 'first_neuron': 16, 'hidden_layers': 3, 'losses': 'logcosh', 'optimizer': 'Nadam', 'shapes': 'brick'}

Params: {'round_epochs': 10, 'val_loss': 0.05940975662734774, 'loss': 0.06377470138527097, 'acc': 0.6666666581517174, 'acti

Now we can access the cross-validation results:

In [12]:
results

,round_epochs,val_loss,val_acc,loss,acc,activation,batch_size,dropout,epochs,first_neuron,hidden_layers,losses,optimizer,shapes,crossval_mean_f1score
8,10,0.032835,0.955556,0.050778,0.761905,elu,20,0.3,10,32,1,logcosh,Nadam,brick,1.000000
2,10,0.026673,0.933333,0.055596,0.714286,elu,50,0.3,10,64,3,logcosh,Nadam,brick,0.565873
5,10,0.017480,0.911111,0.062210,0.752381,elu,40,0.2,10,128,2,logcosh,Nadam,brick,0.682540
4,10,0.037527,0.822222,0.050368,0.761905,relu,20,0.3,10,32,1,logcosh,Nadam,brick,0.933333
9,10,0.069507,0.644444,0.077258,0.628571,relu,50,0.3,10,32,1,logcosh,Nadam,brick,0.533333


We can also access the models and make predictions with them:

In [13]:
models[0].predict(x_val)

array([[0.09669615, 0.6342103 , 0.2690936 ],
       [0.14897366, 0.58802134, 0.26300502],
       [0.94987273, 0.04401869, 0.00610859],
       [0.13425532, 0.588117  , 0.27762768],
       [0.055487  , 0.5218198 , 0.42269325],
       [0.9578523 , 0.03717314, 0.00497459],
       [0.01160751, 0.29222035, 0.6961721 ],
       [0.01371114, 0.40328825, 0.5830006 ],
       [0.9155302 , 0.07383644, 0.01063335],
       [0.12754872, 0.6049956 , 0.26745564],
       [0.02553807, 0.4123017 , 0.56216025],
       [0.01160751, 0.29222035, 0.6961721 ],
       [0.9475036 , 0.04604736, 0.00644898],
       [0.00952294, 0.25774398, 0.7327331 ],
       [0.9716771 , 0.02509327, 0.0032296 ],
       [0.9535259 , 0.04067867, 0.00579546],
       [0.94050807, 0.05207507, 0.00741676],
       [0.01492557, 0.3651738 , 0.61990064],
       [0.00907612, 0.3162972 , 0.6746267 ],
       [0.01725133, 0.3527963 , 0.62995243],
       [0.97003347, 0.0263434 , 0.00362308],
       [0.09504172, 0.56243193, 0.34252638],
       [0.